In [3]:
!ls ../experiments/dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4_mse/results/dev.csv

../experiments/dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4_mse/results/dev.csv


In [4]:
import torch
def CCC_loss(pred, lab, m_lab=None, v_lab=None, is_numpy=True):
    """
    pred: (N, 3)
    lab: (N, 3)
    """
    if is_numpy:
        pred = torch.Tensor(pred).float()
        lab = torch.Tensor(lab).float()
    
    m_pred = torch.mean(pred, 0, keepdim=True)
    m_lab = torch.mean(lab, 0, keepdim=True)

    d_pred = pred - m_pred
    d_lab = lab - m_lab

    v_pred = torch.var(pred, 0, unbiased=False)
    v_lab = torch.var(lab, 0, unbiased=False)

    corr = torch.sum(d_pred * d_lab, 0) / (torch.sqrt(torch.sum(d_pred ** 2, 0)) * torch.sqrt(torch.sum(d_lab ** 2, 0)))

    s_pred = torch.std(pred, 0, unbiased=False)
    s_lab = torch.std(lab, 0, unbiased=False)

    ccc = (2*corr*s_pred*s_lab) / (v_pred + v_lab + (m_pred[0]-m_lab[0])**2)    
    return ccc

In [5]:
import pandas as pd
import numpy as np

In [6]:
preds = pd.read_csv('../experiments/dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4_mse/results/dev.csv')
preds.head()

,Filename,EmoAct_pred,EmoDom_pred,EmoVal_pred
0,MSP-PODCAST_0002_0033.wav,4.056100,4.044851,3.043020
1,MSP-PODCAST_0002_0039.wav,3.617270,3.742541,3.200677
2,MSP-PODCAST_0002_0051.wav,3.427005,3.645125,3.415780
3,MSP-PODCAST_0002_0059.wav,3.822627,3.862643,3.463450
4,MSP-PODCAST_0002_0061.wav,3.524748,3.707217,3.169890


In [7]:
df = pd.read_csv('/workspace/lucas.ueda/interspeech_ser/data/Labels/labels_consensus.csv')
df.head()

,FileName,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_0002_0033.wav,N,4.8,4.2,5.4,127,Female,Development
1,MSP-PODCAST_0002_0039.wav,N,4.0,4.2,4.2,127,Female,Development
2,MSP-PODCAST_0002_0051.wav,N,4.0,4.2,4.2,127,Female,Development
3,MSP-PODCAST_0002_0059.wav,X,4.0,3.8,4.0,128,Female,Development
4,MSP-PODCAST_0002_0061.wav,F,3.4,2.8,4.2,128,Female,Development


In [8]:
preds['FileName'] = preds['Filename']
preds = preds.merge(df[['FileName','EmoAct','EmoVal','EmoDom','Gender','SpkrID']], how='left', on = 'FileName')

In [9]:
# preds['EmoClass'].isnull().sum()

In [10]:
preds.head()

,Filename,EmoAct_pred,EmoDom_pred,EmoVal_pred,FileName,EmoAct,EmoVal,EmoDom,Gender,SpkrID
0,MSP-PODCAST_0002_0033.wav,4.056100,4.044851,3.043020,MSP-PODCAST_0002_0033.wav,4.8,4.2,5.4,Female,127
1,MSP-PODCAST_0002_0039.wav,3.617270,3.742541,3.200677,MSP-PODCAST_0002_0039.wav,4.0,4.2,4.2,Female,127
2,MSP-PODCAST_0002_0051.wav,3.427005,3.645125,3.415780,MSP-PODCAST_0002_0051.wav,4.0,4.2,4.2,Female,127
3,MSP-PODCAST_0002_0059.wav,3.822627,3.862643,3.463450,MSP-PODCAST_0002_0059.wav,4.0,3.8,4.0,Female,128
4,MSP-PODCAST_0002_0061.wav,3.524748,3.707217,3.169890,MSP-PODCAST_0002_0061.wav,3.4,2.8,4.2,Female,128


In [11]:
c = CCC_loss(preds[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds[['EmoAct','EmoVal','EmoDom']].values)

In [12]:
c[0].item(), c[1].item(), c[2].item()

(0.6045978665351868, 0.7128731608390808, 0.5261974334716797)

In [13]:
# Average F1-score using bootstrap
act = []
val = []
dom = []

for i in range(100):
    preds_subset = preds.sample(1000).reset_index(drop=True)
    c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)

    act.append(c[0].item())
    val.append(c[1].item())
    dom.append(c[2].item())

In [14]:
np.mean(act), np.mean(val), np.mean(dom)

(0.6074517786502838, 0.7115796327590942, 0.5291915512084961)

In [15]:
np.std(act), np.std(val), np.std(dom)

(0.020170189004072962, 0.015535349485199986, 0.02261652940041456)

In [16]:
np.min(act), np.min(val), np.min(dom)

(0.5569736361503601, 0.6673967838287354, 0.46635887026786804)

In [17]:
np.max(act), np.max(val), np.max(dom)

(0.6555277109146118, 0.7646738290786743, 0.5797133445739746)

# per gender

In [18]:
preds_subset = preds[preds['Gender'] == 'Female']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.5436, 0.6877, 0.4585])

In [19]:
preds_subset = preds[preds['Gender'] == 'Male']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.6683, 0.7310, 0.6156])